In [282]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import numpy as np

In [283]:
url = "http://fund.eastmoney.com/js/fundcode_search.js" 
response = requests.get(url)
soup = BeautifulSoup(response.content,'lxml')

string = soup.get_text()
pattern = r'\D(\d{6})\D'
fund_code_list = re.findall(pattern,string)  ### 获取市场上所有基金代码

def fund_code(i):
    return fund_code_list[i]


###对单只基金的净值数据进行抓取
def fund_data(i):
    url_i = 'http://fund.eastmoney.com/'+ fund_code(i)+'.html'   
    response_i = requests.get(url_i)
    soup_i = BeautifulSoup(response_i.content,'lxml')

    today = soup_i.find('dl',class_='dataItem01')
    yesterday = soup_i.find('dl',class_='dataItem02')
    summary = soup_i.find('dl',class_='dataItem03')
    
    name = soup_i.find_all('div',class_='SitePath')
    fund_info = soup_i.find_all('div',class_='infoOfFund')
    
    if today == None:
        fund_name = 0
        fund_type = 0
        fund_manager = 0
        zdf_1year = 0
        zdf_1month = 0
        dwjz_today = 0
        zdf_today = 0
        zdf_3year =0
        zdf_3month = 0
        zdf_yesterday = 0
        dwjz_yesterday = 0
        zdf_sincestart =0
        zdf_6month = 0
        ljjz = 0
    else:
        try:
            fund_name = name[0].find_all('a')[2].get_text()
            fund_type = fund_info[0].find_all('a')[0].get_text()
            fund_manager = fund_info[0].find_all('a')[2].get_text()

            data_today = today.find_all('span')
            zdf_1year = data_today[-1].get_text()
            zdf_1month = data_today[-3].get_text()
            dwjz_today = data_today[-7].get_text()
            zdf_today = data_today[-5].get_text()

            data_yesterday = yesterday.find_all('span')
            zdf_3year = data_yesterday[-1].get_text()
            zdf_3month = data_yesterday[-3].get_text()
            zdf_yesterday = data_yesterday[-5].get_text()
            dwjz_yesterday = data_yesterday[-6].get_text()

            data_summary = summary.find_all('span')
            zdf_sincestart = data_summary[-1].get_text()
            zdf_6month = data_summary[-3].get_text()
            ljjz = data_summary[-5].get_text()  
            
        except:   ###货币型基金数据结构 与 股票型、债券型等基金不一致，先暂时剔除
            fund_name = 0
            fund_type = 0
            fund_manager = 0
            zdf_1year = 0
            zdf_1month = 0
            dwjz_today = 0
            zdf_today = 0
            zdf_3year =0
            zdf_3month = 0
            zdf_yesterday = 0
            dwjz_yesterday = 0
            zdf_sincestart =0
            zdf_6month = 0
            ljjz = 0
        
    fund_info = [fund_code(i),fund_name,fund_type,fund_manager,dwjz_today,dwjz_yesterday,ljjz,zdf_today,zdf_1month,zdf_3month,zdf_6month,zdf_1year,zdf_3year,zdf_sincestart] 
    
    return fund_info

In [285]:
result = [fund_data(i) for i in range(len(fund_code_list))]  ### 对市场上所有基金进行遍历，获取每天净值数据

In [293]:
df = pd.DataFrame(result)
df.columns=['基金代码','基金名称','基金类型','基金经理','今日净值','昨日净值','累计净值','今日涨跌幅','过去一月涨跌幅','过去三月涨跌幅',\
            '过去六月涨跌幅','过去一年涨跌幅','过去三年涨跌幅','成立以来涨跌幅',]
df.index = np.arange(1,len(df)+1,1)
df_new = df[df['基金名称']!=0]
df_new.index = np.arange(1,len(df_new)+1,1)
df_new.to_csv('C:\\Users\\lujiani\\Desktop\\东方证券\\珺容\\基金数据库\\20200528',index=False, encoding='utf_8_sig') ###生成excel
df_new.head(10)

,基金代码,基金名称,基金类型,基金经理,今日净值,昨日净值,累计净值,今日涨跌幅,过去一月涨跌幅,过去三月涨跌幅,过去六月涨跌幅,过去一年涨跌幅,过去三年涨跌幅,成立以来涨跌幅
1,000001,华夏成长混合,混合型,董阳阳,1.1789,1.1810,3.6420,-0.18%,2.07%,-1.17%,13.45%,15.77%,22.36%,577.26%
2,000003,中海可转债债券A,债券型,彭海平,0.7455,0.7410,0.9510,+0.61%,-4.63%,-13.43%,0.95%,4.81%,-3.14%,-12.88%
3,000004,中海可转债债券C,债券型,彭海平,0.7454,0.7400,0.9500,+0.73%,-4.76%,-13.45%,0.82%,4.67%,-3.77%,-13.00%
4,000005,嘉实增强信用定期债券,定开债券,刘宁,1.0461,1.0460,1.3880,+0.01%,-0.66%,1.27%,4.46%,8.56%,22.31%,45.42%
5,000006,西部利得量化成长混合,混合型,盛丰衍,1.4108,1.4159,1.4159,-0.36%,4.49%,0.79%,26.32%,46.06%,--,41.59%
6,000008,嘉实中证500ETF联接A,联接基金,何如,1.4442,1.4446,1.5106,-0.03%,2.12%,-6.20%,8.68%,8.17%,-5.27%,51.89%
7,000011,华夏大盘精选混合,混合型,陈伟彦,13.9019,13.8910,18.8310,+0.08%,4.39%,-0.71%,2.48%,9.47%,32.59%,2472.87%
8,000014,华夏聚利债券,债券型,何家琪,1.2996,1.2990,1.2990,+0.05%,-3.13%,-6.01%,5.95%,10.74%,14.35%,29.90%
9,000015,华夏纯债债券A,债券型,柳万军,1.2562,1.2560,1.3690,+0.01%,-0.32%,1.21%,3.06%,5.51%,17.72%,37.97%
10,000016,华夏纯债债券C,债券型,柳万军,1.2242,1.2240,1.3300,+0.01%,-0.33%,1.07%,2.81%,5.07%,16.11%,33.85%
